In [1]:
import sqlite3
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
con = sqlite3.connect("db.sqlite3")
cur = con.cursor()
# read table 
df_store = pd.read_sql_query("SELECT * from api_store", con)
df_review = pd.read_sql_query("SELECT * from api_review", con)

# clonse DB connection
con.close()

In [3]:
# select some columns
stores = df_store[['id','store_name','category']]
reviews = df_review[['user_id','store_id','total_score']]

# rename columns and return data
reviews.rename(columns={'total_score':'rating'}, inplace=True)
stores.rename(columns={'id':'store_id'},inplace=True)

mean_reviews = reviews.groupby(by="user_id", as_index=False)['rating'].mean()
reviews = pd.merge(reviews, mean_reviews, on='user_id')
reviews['adg_rating'] = reviews['rating_x'] - reviews['rating_y']

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [25]:
check = pd.pivot_table(reviews,values='rating_x',index='user_id',columns='store_id')
check.head()

store_id,15,18,19,37,38,49,50,53,56,58,...,360481,360482,360486,360489,360492,360493,360499,360505,360514,360515
user_id,,,,,,,,,,,,,,,,,,,,,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
final = pd.pivot_table(reviews, values='adg_rating', index='user_id', columns='store_id')
final.head()

store_id,15,18,19,37,38,49,50,53,56,58,...,360481,360482,360486,360489,360492,360493,360499,360505,360514,360515
user_id,,,,,,,,,,,,,,,,,,,,,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
final_store = final.fillna(final.mean(axis=0))
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

In [8]:
# user similarity on replacing NAN by user avg
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

user_id,7,15,23,62,64,74,105,137,147,156,...,949436,949554,949596,949670,949944,949946,949951,950224,950225,950331
user_id,,,,,,,,,,,,,,,,,,,,,
7,0.000000e+00,0.0,1.735714e-27,0.0,0.0,0.0,-1.456467e-27,0.0,4.336490e-28,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.675468e-28,0.0,0.0
15,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0
23,1.735714e-27,0.0,0.000000e+00,0.0,0.0,0.0,-4.206993e-28,0.0,1.237613e-28,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.510132e-28,0.0,0.0
62,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0
64,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0


In [16]:
cosine = cosine_similarity(final_store)
np.fill_diagonal(cosine, 0 )
similarity_with_store =pdb.DataFrame(cosine,index=final_store.index)
similarity_with_store.columns=final_user.index
similarity_with_store.head()

NameError: name 'pdb' is not defined

In [9]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [10]:
# top 30 neighbours for each user
sim_user_30_u = find_n_neighbours(similarity_with_user,30)
sim_user_30_u.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
user_id,,,,,,,,,,,,,,,,,,,,,
7,318374,150785,257535,46985,95895,199961,49575,713971,360576,153765,...,322822,131826,173838,174506,192820,293204,783901,364434,310432,779112
15,950331,179160,179170,179191,179192,179262,179299,179325,179341,179346,...,179121,178854,178886,178898,178908,178924,178978,178985,178986,179008
23,86373,81687,217426,177837,550410,73273,70651,650646,312492,485567,...,64397,514754,46985,95895,49575,199961,153765,360576,713971,256446
62,950331,179160,179170,179191,179192,179262,179299,179325,179341,179346,...,179121,178854,178886,178898,178908,178924,178978,178985,178986,179008
64,950331,179160,179170,179191,179192,179262,179299,179325,179341,179346,...,179121,178854,178886,178898,178908,178924,178978,178985,178986,179008


In [19]:
# top 30 neighbours for each user
sim_user_30_m = find_n_neighbours(similarity_with_store,30)
sim_user_30_m.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
user_id,,,,,,,,,,,,,,,,,,,,,
7,486476,290816,169172,616215,345576,80469,616578,345537,616746,169119,...,617428,616106,169115,344465,80822,614251,81974,81812,614697,614750
15,290816,169418,616106,616215,169119,80469,345576,616578,616746,345537,...,617428,80612,169115,80822,169016,614251,81974,81812,614697,614750
23,81687,940419,650994,219739,129845,138495,621365,118858,118704,455454,...,457058,120288,457632,457992,458620,117553,117535,117386,117381,460378
62,290816,344465,616215,80612,169149,80469,616578,345576,616746,345537,...,617501,616106,169119,80822,615118,346977,81974,614697,614750,346831
64,290816,344465,616215,80612,169149,80469,616578,345576,616746,345537,...,617501,616106,169119,80822,615118,346977,81974,614697,614750,346831


In [33]:
reviews = reviews.astype({"store_id": str})
Movie_user = reviews.groupby(by = 'user_id')['store_id'].apply(lambda x:','.join(x))

In [34]:
def User_item_score1(user):
    Movie_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = Movie_user[Movie_user.index.isin(b)]
    l = ','.join(d.values)
    Movie_seen_by_similar_users = l.split(',')
    Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
    Movies_under_consideration = list(map(int, Movies_under_consideration))
    score = []
    for item in Movies_under_consideration:
        c = final_store.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = mean_reviews.loc[mean_reviews['user_id'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_store.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'store_id':Movies_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    Movie_Name = top_5_recommendation.merge(stores, how='inner', on='store_id')
    print(Movie_Name)
    Movie_Names = Movie_Name.store_name.values.tolist()
    return Movie_Names

In [35]:
user = int(input("Enter the user id to whom you want to recommend : "))
predicted_movies = User_item_score1(user)
print(" ")
print("The Recommendations for User Id :"+str(user))
print("   ")
for i in predicted_movies:
    print(i)

Enter the user id to whom you want to recommend : 4169


ValueError: invalid literal for int() with base 10: ''

In [5]:
import sqlite3
import pandas as pd
import numpy as np
import sklearn
import surprise

In [6]:
con = sqlite3.connect("db.sqlite3")
cur = con.cursor()

In [7]:
df_store = pd.read_sql_query("SELECT * FROM api_store", con)
df_review = pd.read_sql_query("SELECT * FROM api_review", con)

In [8]:
df_store.head()

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score
0,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차,0,0
1,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페,0,0
2,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍,0,0
3,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제,0,0
4,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트,0,0


In [9]:
df_review.head()

,id,total_score,content,reg_time,store_id,user_id
0,1,5.0,전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!,2020-04-07 07:13:49.444324,15,68632
1,2,5.0,샌드위치 내용물도 알차게 들어있고 맛있었어요 가성비 추천,2020-04-07 07:13:49.444324,18,389728
2,3,4.0,홈플러스 1층 매장 푸드코트 내 있는 매장인데 계란찜정식은 처음보고 시켜봣는데 사진...,2020-04-07 07:13:49.444324,19,68716
3,4,2.0,"전 여기 5년전에 가봤었는데 그때 기억은 별로였어요\n단체손님이 많았고, 차려지는건...",2020-04-07 07:13:49.444324,37,774353
4,5,3.0,친구들끼리 술 간단하게마시러 감. 스끼다시 괜찮지만 회 양이 조금 부족한 느낌. 맛...,2020-04-07 07:13:49.444324,38,115682


In [10]:
con.close()

In [ ]:
stores = df_store[['id', 'store_name', 'category', 'address']]
reviews = df_review[['user_id', 'store_id', 'total_score']]
stores.rename(colums={})
reviews.rename(colums={'total_score': 'rating'}, inplace=True)